In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

import optuna

Using TensorFlow backend.


In [2]:
df = pd.read_csv('does_it_work.csv')
pd.set_option('display.max_columns', 110)
df

,AGEP,CIT,DEYE,MAR,SEX,ANC,NATIVITY,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACPI,RACSOR,DOUT_cat,FER_cat,INTP_cat,LANX_cat,MARHW_cat,MIL_cat,MLPA_cat,OIP_cat,RETP_cat,SCHG_cat,SCHL_cat,SEMP_cat,SSIP_cat,WKL_cat,WKW_cat,WRK_cat,YOEP_cat,DRIVESP_cat,ESR_cat,NAICSP_cat,POBP_cat,POWSP_cat,SCIENGRLP_cat,JWAP_mins,JWDP_mins,TOTAL_commute,PWGTP_y,CITWP,COW,DDRS,DEAR,DPHY,DREM,ENG,JWMNP,JWRIP,JWTR,MARHD,MARHM,MARHT,MIG,NWLK,PAP,RELP,SCH,SSP,WAGP,WKHP,DECADE,DIS,ESP,HISP,INDP,MSP,NOP,OC,OCCP,PERNP,PINCP,QTRBIR,RACWHT,RC,SCIENGP,VPS,WAOB,CIT_STATUS,BORN_CIT,HICOV,NP,TYPE,FINCP,HINCP,RACES,MIL_STAT,INDUSTRY,SOCP_recat,MAR_YEARS
0,26,5,2,Never Married,Female,1,2,0,0,0,0,1,0,0,No,No,0.0,Other_Lang,Unknown,Never Served,LT_17yrold,0.0,0.0,Graduate,Bachelors,0.0,0.0,LT1,LT_14wks,Not_Reported,Immigr_10s,No_Drive,Not_Labor,EDU,0,0,No,0.0,0.0,0.0,145,1.0,2.0,2.0,2,2.0,2.0,Very Well,201.0,0.0,14.0,3.0,3.0,3.0,1.0,4.0,0.0,17,3.0,0.0,5200.0,50.0,8.0,Not Disabled,0.0,1,EDU,6.0,0.0,0.0,5110.0,5200.0,5200.0,4,1,3.0,1.0,16.0,5,0,0,1,1,3,0.0,0.0,White,Served,Education,OFF,0.0
1,21,1,2,Never Married,Male,1,1,0,0,0,0,1,0,0,No,Not_Applicable,0.0,English_Lang,Unknown,Never Served,LT_17yrold,0.0,0.0,Undergrad,Associates,0.0,0.0,LT1,14_26wks,Worked,Naturalized,No_Drive,Employed_at_work,ENT2,1,1,LT_Bachelors,1425.0,1410.0,15.0,43,0.0,1.0,2.0,2,2.0,2.0,Only English,2.0,0.0,10.0,3.0,3.0,3.0,3.0,4.0,0.0,17,2.0,0.0,1000.0,2.0,0.0,Not Disabled,0.0,1,ENT2,6.0,0.0,0.0,4110.0,1000.0,1000.0,2,1,3.0,0.0,16.0,1,1,1,1,1,3,0.0,0.0,White,Served,Entertainment,EAT,0.0
2,18,2,2,Never Married,Female,1,1,0,0,0,0,1,0,1,No,No,0.0,Other_Lang,Unknown,Never Served,LT_17yrold,0.0,0.0,Undergrad,HS_Diploma,0.0,0.0,BT1_5,Not_Worked,Not_Worked,Immigr_00s,No_Drive,Not_Labor,ENT2,0,0,LT_Bachelors,0.0,0.0,0.0,88,0.0,1.0,2.0,2,2.0,2.0,Very Well,201.0,0.0,14.0,3.0,3.0,3.0,3.0,4.0,0.0,17,2.0,0.0,0.0,0.0,0.0,Not Disabled,0.0,2,ENT2,6.0,0.0,0.0,4720.0,0.0,0.0,3,0,3.0,0.0,16.0,2,1,1,1,1,3,0.0,0.0,Other,Served,Entertainment,SAL,0.0
3,85,1,2,Widowed,Female,4,1,0,0,0,0,1,0,0,No,Not_Applicable,0.0,English_Lang,2.0,Never Served,LT_17yrold,0.0,0.0,Not_Attending,HS_Diploma,0.0,0.0,GT5,Not_Worked,Not_Reported,Naturalized,No_Drive,Not_Labor,No_Work,1,0,LT_Bachelors,0.0,0.0,0.0,109,0.0,10.0,1.0,1,1.0,1.0,Only English,201.0,0.0,14.0,2.0,2.0,2.0,1.0,4.0,0.0,16,1.0,13700.0,0.0,0.0,0.0,Disabled,0.0,1,None,3.0,0.0,0.0,1.0,0.0,13700.0,3,1,3.0,0.0,16.0,1,1,1,1,1,2,0.0,0.0,White,Served,None,None,68.0
4,19,1,2,Never Married,Female,2,1,0,0,0,0,1,0,0,No,No,0.0,English_Lang,Unknown,Never Served,LT_17yrold,0.0,0.0,Undergrad,NoDegree_College,0.0,0.0,LT1,14_26wks,Worked,Naturalized,5_6_driver,Employed_at_work,ENT1,1,1,LT_Bachelors,915.0,910.0,5.0,83,0.0,1.0,2.0,2,2.0,2.0,Only English,5.0,5.0,1.0,3.0,3.0,3.0,1.0,4.0,0.0,17,3.0,0.0,870.0,4.0,0.0,Not Disabled,0.0,1,ENT1,6.0,0.0,0.0,2723.0,870.0,870.0,4,1,3.0,0.0,16.0,1,1,1,1,1,3,0.0,0.0,White,Served,Entertainment,ENT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159294,36,1,2,Married,Female,1,1,0,0,0,0,1,0,0,No,No,0.0,English_Lang,2.0,Never Served,LT_17yrold,0.0,0.0,Not_Attending,Masters,0.0,0.0,LT1,50_52wks,Worked,Naturalized,No_Drive,Employed_at_work,INF,1,1,No,525.0,475.0,50.0,113,0.0,1.0,2.0,2,2.0,2.0,Only English,45.0,0.0,4.0,2.0,2.0,2.0,1.0,4.0,0.0,0,1.0,0.0,90000.0,40.0,0.0,Not Disabled,0.0,1,INF1,1.0,0.0,0.0,1021.0,90000.0,90000.0,1,1,0.0,2.0,16.0,1,1,1,1,3,1,220000.0,220000.0,White,Served,Information,CMM,7.0
159295,38,1,2,Married,Male,1,1,0,1,0,0,1,0,0,No,Not_Applicable,0.0,Other_Lang,2.0,Never Served,LT_17yrold,0.0,0.0,Not_Attending,Masters,0.0,0.0,LT1,50_52wks,Worked,Naturalized,No_Drive,Employed_at_work,PRF1,0,1,No,520.0,475.0,45.0,129,0.0,1.0,2.0

In [3]:
df['HICOV'].value_counts()

1    151320
0      7979
Name: HICOV, dtype: int64

In [4]:
X = df.drop(columns=['HICOV'])
y = df['HICOV']

In [5]:
continuous_columns = ['AGEP','INTP_cat','OIP_cat','RETP_cat','SEMP_cat','SSIP_cat','JWAP_mins','JWDP_mins','TOTAL_commute','PWGTP_y','JWMNP','PAP','SSP','WAGP','WKHP','OCCP','PERNP','PINCP',
                     'FINCP','HINCP','MAR_YEARS','NP']
categorical_columns = [x for x in X.columns if x not in continuous_columns]

In [6]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ohe = OneHotEncoder()
_X_train_ohe = ohe.fit_transform(X_train[categorical_columns]).toarray()
_X_train_ohe_df = pd.DataFrame(data = _X_train_ohe,
                        columns = ohe.get_feature_names(categorical_columns))

_X_test_ohe = ohe.transform(X_test[categorical_columns]).toarray()
_X_test_ohe_df = pd.DataFrame(data = _X_test_ohe,
                        columns = ohe.get_feature_names(categorical_columns))

In [7]:

scaler = StandardScaler()
_X_train_cont = scaler.fit_transform(X_train[continuous_columns])
_X_train_cont_df = pd.DataFrame(data = _X_train_cont,
                         columns = continuous_columns)

_X_test_cont = scaler.transform(X_test[continuous_columns])
_X_test_cont_df = pd.DataFrame(data = _X_test_cont,
                         columns = continuous_columns)

In [8]:
_X_test_merged = _X_test_cont_df[continuous_columns].merge(_X_test_ohe_df, left_index=True, right_index=True)
_X_train_merged = _X_train_cont_df[continuous_columns].merge(_X_train_ohe_df, left_index=True, right_index=True)
_X_train_merged

,AGEP,INTP_cat,OIP_cat,RETP_cat,SEMP_cat,SSIP_cat,JWAP_mins,JWDP_mins,TOTAL_commute,PWGTP_y,JWMNP,PAP,SSP,WAGP,WKHP,OCCP,PERNP,PINCP,FINCP,HINCP,MAR_YEARS,NP,CIT_1,CIT_2,CIT_3,CIT_4,CIT_5,DEYE_1,DEYE_2,MAR_Divorced,MAR_Married,MAR_Never Married,MAR_Separated,MAR_Widowed,SEX_Female,SEX_Male,ANC_1,ANC_2,ANC_3,ANC_4,NATIVITY_1,NATIVITY_2,RACAIAN_0,RACAIAN_1,RACASN_0,RACASN_1,RACBLK_0,RACBLK_1,RACNH_0,RACNH_1,RACNUM_1,RACNUM_2,RACNUM_3,RACNUM_4,RACNUM_5,...,WAOB_7,WAOB_8,CIT_STATUS_0,CIT_STATUS_1,BORN_CIT_0,BORN_CIT_1,TYPE_1,TYPE_2,TYPE_3,RACES_American Indigenous,RACES_Asian,RACES_Black,RACES_More Than One Race,RACES_Other,RACES_White,MIL_STAT_Served,INDUSTRY_Administration,INDUSTRY_Education,INDUSTRY_Entertainment,INDUSTRY_Finance,INDUSTRY_Information,INDUSTRY_Manufacturing,INDUSTRY_Medical,INDUSTRY_Military,INDUSTRY_None,INDUSTRY_Professional,INDUSTRY_Retail,INDUSTRY_Service,INDUSTRY_Social Service,INDUSTRY_Transportation,INDUSTRY_Wholesale,SOCP_recat_BUS,SOCP_recat_CLN,SOCP_recat_CMM,SOCP_recat_CMS,SOCP_recat_EAT,SOCP_recat_EDU,SOCP_recat_ENG,SOCP_recat_ENT,SOCP_recat_EXT,SOCP_recat_FFF,SOCP_recat_FIN,SOCP_recat_HLS,SOCP_recat_LGL,SOCP_recat_MED,SOCP_recat_MGR,SOCP_recat_MIL,SOCP_recat_None,SOCP_recat_OFF,SOCP_recat_PRD,SOCP_recat_PRS,SOCP_recat_PRT,SOCP_recat_SAL,SOCP_recat_SCI,SOCP_recat_TRN
0,0.962404,0.319058,-0.135018,-0.245121,5.078202,-0.169988,0.248737,0.175238,0.895517,-0.468177,-0.746768,-0.09087,-0.487801,-0.521691,1.231343,0.666702,0.862692,0.793119,0.432761,0.284290,1.481046,-0.513751,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.486627,-0.125129,-0.135018,-0.245121,-0.111339,-0.169988,0.769205,0.739064,0.713803,0.167690,-0.804819,-0.09087,-0.257177,0.181585,0.513204,-0.219882,0.143615,0.017352,-0.728598,0.449802,2.000353,-0.513751,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.857560,-0.118910,-0.135018,-0.245121,-0.111339,-0.169988,0.557765,0.482779,1.077230,-0.701761,-0.688716,-0.09087,-0.487801,0.089853,0.752584,0.819194,0.055990,-0.084274,0.244934,0.097633,1.429115,0.051066,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.805138,0.407895,-0.135018,0.781440,-0.111339,-0.169988,0.996910,0.585293,4.893208,-0.870461,0.391044,-0.09087,-0.487801,0.861564,0.991963,-0.327691,0.793149,0.928878,-0.728598,0.088094,-0.959699,-1.078568,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.396598,-0.125129,0.433421,-0.245121,-0.111339,-0.169988,-0.954845,-0.935328,-0.739903,0.816535,1.122495,-0.09087,-0.487801,-0.294546,0.178073,0.418458,-0.311199,-0.391219,0.895289,0.743938,-0.959699,0.615883,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,

In [9]:

_smote = SMOTE(sampling_strategy='minority', random_state=42)
_X_train_merged, y_train = _smote.fit_sample(_X_train_merged, y_train)

In [10]:

bc_lr1 =LogisticRegression(solver='liblinear', penalty='l1')

# {'lr_C': 0.04108295837286748, 'lr_solver': 'saga'}
bc_lr1.fit(_X_train_merged, y_train)
y_train_pred = bc_lr1.predict(_X_train_merged)
y_test_pred = bc_lr1.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.8100543885475525
Testing Precision:  0.9835383889912976



Training Recall:  0.7593749483530832
Testing Recall:  0.7570448096086584



Training Accuracy:  0.7906564529723834
Testing Accuracy:  0.756842435655995



Training F1-Score:  0.7838964074435824
Testing F1-Score:  0.8555553483862548


In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)


rfc = RandomForestClassifier(max_depth=5,
                            max_leaf_nodes=39,
                            max_features = .2479,
                            min_samples_leaf=8,
                            max_samples = 0.2647,
                            criterion='entropy')


# {'rf_max_depth': 5, 'rf_max_leaf_nodes': 39, 'rf_max_features': 0.24795749696390704, 'rf_min_samples_leaf': 8, 'rf_max_samples': 0.26471291186578816, 'rf_criterion': 'entropy'}
# parameters = {'max_leaf_nodes': range(10,100,1)}
#     'min_samples_split': range(2,10,1)}
#     'n_estimators': range(100, 500, 50)}
#     'max_depth': range(10,25,1)}
#              'min_samples_leaf': range(1,10,1),
#              'class_weight': ['balanced',None,'balanced_subsample']}
# grid_tree = GridSearchCV(rfc, parameters, cv=5, scoring='f1', n_jobs=-1,verbose=1)

rfc.fit(_X_train_merged, y_train)
y_train_pred = rfc.predict(_X_train_merged)
y_test_pred = rfc.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.8676242715770061
Testing Precision:  0.9784409153005464



Training Recall:  0.7541937296511148
Testing Recall:  0.7562528872170527



Training Accuracy:  0.8195621994149437
Testing Accuracy:  0.7522912743251726



Training F1-Score:  0.806942313898332
Testing F1-Score:  0.8531174390470874


In [12]:
_coef_df = pd.DataFrame()
_coef_df['features'] = _X_train_merged.columns
_coef_df['coef'] = bc_lr1.coef_[0]
_coef_df

,features,coef
0,AGEP,0.180637
1,INTP_cat,-0.093139
2,OIP_cat,0.004542
3,RETP_cat,0.388032
4,SEMP_cat,-0.126022
...,...,...
461,SOCP_recat_PRS,0.207727
462,SOCP_recat_PRT,0.293574
463,SOCP_recat_SAL,-0.036057
464,SOCP_recat_SCI,1.710781


In [13]:
import optuna

def objective(trial):
    rf_max_depth = trial.suggest_int('rf_max_depth', 18, 54)
    rf_max_leaf_nodes = trial.suggest_int('rf_max_leaf_nodes', 54, 96)
    rf_max_features = trial.suggest_loguniform('rf_max_features', .1, .9)
    rf_min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 2, 20)
    rf_max_samples = trial.suggest_loguniform('rf_max_samples', .1, .9)
    rf_criterion = trial.suggest_categorical('rf_criterion', ['gini', 'entropy'])
    
    regressor_obj = RandomForestClassifier(max_depth = rf_max_depth,
                                          max_leaf_nodes = rf_max_leaf_nodes,
                                          max_features = rf_max_features,
                                          min_samples_leaf = rf_min_samples_leaf,
                                          max_samples = rf_max_samples,
                                          n_estimators=300,
                                          random_state=42,
                                          criterion = rf_criterion)
    
    regressor_obj.fit(_X_train_merged, y_train)
    y_pred = regressor_obj.predict(_X_test_merged)
    
    return (1 - recall_score(y_test, y_pred))

In [14]:
# study = optuna.create_study()
# study.optimize(objective, n_trials=100)

In [15]:


rfc = RandomForestClassifier(max_depth=27,
                            max_leaf_nodes=81,
                            max_features = .40431566940142366,
                            min_samples_leaf=14,
                            max_samples = 0.24057939625788344,
                            criterion='entropy')

# {'rf_max_depth': 27, 'rf_max_leaf_nodes': 81, 'rf_max_features': 0.40431566940142366, 'rf_min_samples_leaf': 14, 'rf_max_samples': 0.24057939625788344, 'rf_criterion': 'entropy'}
#{'rf_max_depth': 18, 'rf_max_leaf_nodes': 64, 'rf_max_features': 0.3308894632219787, 'rf_min_samples_leaf': 10, 'rf_max_samples': 0.10496753020075641, 'rf_criterion': 'gini'}
# {'rf_max_depth': 5, 'rf_max_leaf_nodes': 39, 'rf_max_features': 0.24795749696390704, 'rf_min_samples_leaf': 8, 'rf_max_samples': 0.26471291186578816, 'rf_criterion': 'entropy'}
# parameters = {'max_leaf_nodes': range(10,100,1)}
#     'min_samples_split': range(2,10,1)}
#     'n_estimators': range(100, 500, 50)}
#     'max_depth': range(10,25,1)}
#              'min_samples_leaf': range(1,10,1),
#              'class_weight': ['balanced',None,'balanced_subsample']}
# grid_tree = GridSearchCV(rfc, parameters, cv=5, scoring='f1', n_jobs=-1,verbose=1)

rfc.fit(_X_train_merged, y_train)
y_train_pred = rfc.predict(_X_train_merged)
y_test_pred = rfc.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9058542568659784
Testing Precision:  0.9706301484133328



Training Recall:  0.9215214768539177
Testing Recall:  0.9214676961657757



Training Accuracy:  0.9128737170905845
Testing Accuracy:  0.8987758945386064



Training F1-Score:  0.9136207044867095
Testing F1-Score:  0.9454102273304332


In [16]:
evaluation_results = {}
clf = LGBMClassifier(objective='binary',
                     boosting='gbdt',
                     learning_rate = 0.05,
                     max_depth = 8,
                     num_leaves = 80,
                     n_estimators = 400,
                     bagging_fraction = 0.8,
                     feature_fraction = 0.9)


clf.fit(_X_train_merged, y_train)
y_train_pred = clf.predict(_X_train_merged)
y_test_pred = clf.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(y_train, y_train_pred))
print('Testing Precision: ', precision_score(y_test, y_test_pred))
print('\n\n')

print('Training Recall: ', recall_score(y_train, y_train_pred))
print('Testing Recall: ', recall_score(y_test, y_test_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_train, y_train_pred))
print('Testing Accuracy: ', accuracy_score(y_test, y_test_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(y_train, y_train_pred))
print('Testing F1-Score: ', f1_score(y_test, y_test_pred))

Training Precision:  0.9606750085731831
Testing Precision:  0.959389731175946



Training Recall:  0.995422017287256
Testing Recall:  0.9938955982313733



Training Accuracy:  0.977337332870577
Testing Accuracy:  0.9541745134965474



Training F1-Score:  0.9777398997585276
Testing F1-Score:  0.9763378820783766


In [17]:
import pickle
with open('lgbm_classifier.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [18]:
clf

LGBMClassifier(bagging_fraction=0.8, boosting='gbdt', boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.05, max_depth=8,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, num_leaves=80, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
_X = X.copy()
_y = y.copy()

In [20]:
ohe_final = OneHotEncoder()
_X_final_ohe = ohe_final.fit_transform(_X[categorical_columns]).toarray()
_X_final_ohe_df = pd.DataFrame(data = _X_final_ohe,
                        columns = ohe_final.get_feature_names(categorical_columns))

In [21]:
from sklearn.preprocessing import StandardScaler
scaler_final = StandardScaler()
_X_final_cont = scaler_final.fit_transform(_X[continuous_columns])
_X_final_cont_df = pd.DataFrame(data = _X_final_cont,
                         columns = continuous_columns)


In [22]:
# _X_test_merged = _X_test_cont_df[continuous_columns].merge(_X_test_ohe_df, left_index=True, right_index=True)
_X_final_merged = _X_final_cont_df[continuous_columns].merge(_X_final_ohe_df, left_index=True, right_index=True)
_X_final_merged

,AGEP,INTP_cat,OIP_cat,RETP_cat,SEMP_cat,SSIP_cat,JWAP_mins,JWDP_mins,TOTAL_commute,PWGTP_y,JWMNP,PAP,SSP,WAGP,WKHP,OCCP,PERNP,PINCP,FINCP,HINCP,MAR_YEARS,NP,CIT_1,CIT_2,CIT_3,CIT_4,CIT_5,DEYE_1,DEYE_2,MAR_Divorced,MAR_Married,MAR_Never Married,MAR_Separated,MAR_Widowed,SEX_Female,SEX_Male,ANC_1,ANC_2,ANC_3,ANC_4,NATIVITY_1,NATIVITY_2,RACAIAN_0,RACAIAN_1,RACASN_0,RACASN_1,RACBLK_0,RACBLK_1,RACNH_0,RACNH_1,RACNUM_1,RACNUM_2,RACNUM_3,RACNUM_4,RACNUM_5,...,WAOB_7,WAOB_8,CIT_STATUS_0,CIT_STATUS_1,BORN_CIT_0,BORN_CIT_1,TYPE_1,TYPE_2,TYPE_3,RACES_American Indigenous,RACES_Asian,RACES_Black,RACES_More Than One Race,RACES_Other,RACES_White,MIL_STAT_Served,INDUSTRY_Administration,INDUSTRY_Education,INDUSTRY_Entertainment,INDUSTRY_Finance,INDUSTRY_Information,INDUSTRY_Manufacturing,INDUSTRY_Medical,INDUSTRY_Military,INDUSTRY_None,INDUSTRY_Professional,INDUSTRY_Retail,INDUSTRY_Service,INDUSTRY_Social Service,INDUSTRY_Transportation,INDUSTRY_Wholesale,SOCP_recat_BUS,SOCP_recat_CLN,SOCP_recat_CMM,SOCP_recat_CMS,SOCP_recat_EAT,SOCP_recat_EDU,SOCP_recat_ENG,SOCP_recat_ENT,SOCP_recat_EXT,SOCP_recat_FFF,SOCP_recat_FIN,SOCP_recat_HLS,SOCP_recat_LGL,SOCP_recat_MED,SOCP_recat_MGR,SOCP_recat_MIL,SOCP_recat_None,SOCP_recat_OFF,SOCP_recat_PRD,SOCP_recat_PRS,SOCP_recat_PRT,SOCP_recat_SAL,SOCP_recat_SCI,SOCP_recat_TRN
0,-1.239857,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,0.619286,1.123548,-0.090197,-0.486672,-0.448546,1.230394,0.772516,-0.457984,-0.564065,-0.730566,-0.873149,-0.959950,-1.078661,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.502367,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,3.677445,3.879619,-0.196193,-0.701801,-1.187609,-0.090197,-0.486672,-0.510080,-1.068971,0.417765,-0.516662,-0.618578,-0.730566,-0.873149,-0.959950,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.659874,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,-0.118968,1.123548,-0.090197,-0.486672,-0.524730,-1.164778,0.634163,-0.530633,-0.631557,-0.730566,-0.873149,-0.959950,-1.078661,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.857764,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,-0.955794,-0.936232,-0.740011,0.153020,1.123548,-0.090197,1.372175,-0.524730,-1.164778,-1.039910,-0.530633,-0.453740,-0.730566,-0.873149,2.571079,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.607372,-0.125537,-0.134285,-0.245213,-0.111282,-0.169387,2.019233,2.171870,-0.558738,-0.183727,-1.152767,-0.090197,-0.486672,-0.511984,-0.973165,-0.074276,-0.518478,-0.620265,-0.730566,-0.873149,-0.959950,-1.078661,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0

In [23]:
from imblearn.over_sampling import SMOTE
_smote_final = SMOTE(sampling_strategy='minority', random_state=42)
_X_final_merged, _y = _smote_final.fit_sample(_X_final_merged, _y)

In [24]:

clf_final = LGBMClassifier(objective='binary',
                     boosting='gbdt',
                     learning_rate = 0.05,
                     max_depth = 8,
                     num_leaves = 80,
                     n_estimators = 400,
                     bagging_fraction = 0.8,
                     feature_fraction = 0.9)


clf_final.fit(_X_final_merged, _y)
y_final_pred = clf_final.predict(_X_final_merged)
# y_test_pred = clf.predict(_X_test_merged)


from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print('Training Precision: ', precision_score(_y, y_final_pred))
print('\n\n')

print('Training Recall: ', recall_score(_y, y_final_pred))
print('\n\n')

print('Training Accuracy: ', accuracy_score(_y, y_final_pred))
print('\n\n')

print('Training F1-Score: ', f1_score(_y, y_final_pred))

Training Precision:  0.9605910387199265



Training Recall:  0.9950039651070579



Training Accuracy:  0.9770915939730372



Training F1-Score:  0.9774947169554081


In [25]:
import pickle
with open('lgbm_final_classifier.pickle', 'wb') as f:
    pickle.dump(clf_final, f)

In [26]:
ohe_final

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [27]:
with open('ohe_final.pickle', 'wb') as f:
    pickle.dump(ohe_final, f)

In [28]:
scaler_final

StandardScaler(copy=True, with_mean=True, with_std=True)

In [29]:
with open('scaler_final.pickle', 'wb') as f:
    pickle.dump(scaler_final, f)

In [30]:
_coef_df.to_csv('logreg_coefficients.csv', index=False, header=True)